In [27]:
#!pip install polaris-ml
#!pip install fets
#!pip install mlflow
#!pip install --upgrade pip
#!pip install causalnex
#!python -c "import causalnex"
#!pip install torch
#!pip install pygraphviz

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pygraphviz
Failed to build pygraphviz
    Running setup.py install for pygraphviz: started
    Running setup.py install for pygraphviz: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\WPy64-3920\python-3.9.2.amd64\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\manni\\AppData\\Local\\Temp\\pip-install-vq5ybz52\\pygraphviz_45fbd42973724c6e82dcd5b50b94800d\\setup.py'"'"'; __file__='"'"'C:\\Users\\manni\\AppData\\Local\\Temp\\pip-install-vq5ybz52\\pygraphviz_45fbd42973724c6e82dcd5b50b94800d\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\manni\AppData\Local\Temp\pip-wheel-816rc8g7'
       cwd: C:\Users\manni\AppData\Local\Temp\pip-install-vq5ybz52\pygraphviz_45fbd42973724c6e82dcd5b50b94800d\
  Complete output (47 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.wi

Ok


Ok


In [21]:

import warnings
from causalnex.structure import StructureModel

warnings.filterwarnings("ignore")  # silence warnings

sm = StructureModel()

print("Ok")


Ok


In [22]:

sm.add_edges_from([
    ('health', 'absences'),
    ('health', 'G1')
])

print("Ok")


Ok


In [23]:
sm.edges

OutEdgeView([('health', 'absences'), ('health', 'G1')])

In [ ]:
from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
Image(viz.draw(format='png'))

In [4]:
import pandas as pd

data = pd.read_csv('student-por.csv', delimiter=';')
data.head(5)

Ok


In [6]:
drop_col = ['school','sex','age','Mjob', 'Fjob','reason','guardian']
data = data.drop(columns=drop_col)
data.head(5)

SyntaxError: EOL while scanning string literal (<ipython-input-6-9e4c8a80f6fa>, line 2)

In [ ]:
import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

170

(239, 3)

In [ ]:
from causalnex.structure.notears import from_pandas

sm = from_pandas(struct_data)

print("Ok")

In [ ]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
    prog='fdp',
)
Image(viz.draw(format='png'))

In [ ]:

sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
Image(viz.draw(format='png'))

In [ ]:
sm = from_pandas(struct_data, tabu_edges=[("higher", "Medu")], w_threshold=0.8)
print("Ok")

In [ ]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
Image(viz.draw(format='png'))